<a href="https://colab.research.google.com/github/equitymarkets/health_project_group_1/blob/main/hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


Raw Data

In [ ]:
med_conditions = pd.read_sas('/content/P_MCQ.XPT', format = 'xport')
med_conditions.head()

,SEQN,MCQ010,MCQ025,MCQ035,MCQ040,MCQ050,AGQ030,MCQ053,MCQ080,MCQ092,...,MCQ300A,MCQ366A,MCQ366B,MCQ366C,MCQ366D,MCQ371A,MCQ371B,MCQ371C,MCQ371D,OSQ230
0,109263.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109265.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,1.0,9.0,...,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN
4,109267.0,2.0,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,NaN


In [ ]:
columns = ['SEQN', 'MCQ010', 'MCQ160B', 'MCQ160C', 'MCQ160D', 'MCQ160E', 'MCQ160F', 'MCQ160M', 'MCQ500', 'MCQ560']
med_conditions_reduced = med_conditions[columns]
med_conditions_reduced.head()

,SEQN,MCQ010,MCQ160B,MCQ160C,MCQ160D,MCQ160E,MCQ160F,MCQ160M,MCQ500,MCQ560
0,109263.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,109265.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0
4,109267.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0


In [ ]:
mc_df = med_conditions_reduced.rename(columns = {'MCQ010': 'asthma', 'MCQ160B': 'heart_failure', 
                                                 'MCQ160C': 'heart_disease', 'MCQ160D': 'angina', 
                                                 'MCQ160E': 'heart_attack', 'MCQ160F': 'stroke', 
                                                 'MCQ160M': 'thyroid', 'MCQ500': 'liver', 
                                                 'MCQ560': 'gallbladder', 'SEQN': 'id'})
mc_df.head()

,id,asthma,heart_failure,heart_disease,angina,heart_attack,stroke,thyroid,liver,gallbladder
0,109263.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,109265.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0
4,109267.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0


In [ ]:
mc_df['asthma'] = mc_df['asthma'].fillna(0)
mc_df['heart_failure'] = mc_df['heart_failure'].fillna(0)
mc_df['heart_disease'] = mc_df['heart_disease'].fillna(0)
mc_df['angina'] = mc_df['angina'].fillna(0)
mc_df['heart_attack'] = mc_df['heart_attack'].fillna(0)
mc_df['stroke'] = mc_df['stroke'].fillna(0)
mc_df['thyroid'] = mc_df['thyroid'].fillna(0)
mc_df['liver'] = mc_df['liver'].fillna(0)
mc_df['gallbladder'] = mc_df['gallbladder'].fillna(0)

mc_df.head()

,id,asthma,heart_failure,heart_disease,angina,heart_attack,stroke,thyroid,liver,gallbladder
0,109263.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,109264.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
2,109265.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109266.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0
4,109267.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,2.0


In [ ]:
mc_df['asthma'].nunique()

3

In [ ]:
index_drop = list(mc_df[ (mc_df['asthma']==9) | (mc_df['heart_failure']==9) | (mc_df['heart_disease']==9) | (mc_df['angina']==9) | (mc_df['heart_attack']==9) | (mc_df['stroke']==9) | (mc_df['thyroid']==9) | (mc_df['liver']==9) | (mc_df['gallbladder']==9) ].index)


In [ ]:
print(index_drop)

[309, 359, 421, 426, 798, 1103, 1122, 1168, 1490, 2192, 2248, 2302, 2348, 2361, 2710, 2982, 3015, 3101, 3211, 3241, 3353, 3592, 3593, 3729, 4091, 4099, 4125, 4321, 4446, 4530, 4545, 4655, 4837, 4911, 4989, 5020, 5277, 5306, 5312, 5351, 5568, 5608, 5612, 5677, 5695, 5799, 5855, 5859, 5929, 5932, 5950, 5963, 6202, 6362, 6399, 6592, 6606, 6608, 6834, 6943, 6944, 7232, 7590, 7633, 7659, 8009, 8080, 8303, 8385, 8602, 8843, 8856, 9134, 9433, 9438, 9591, 9609, 9681, 9782, 9831, 9976, 10026, 10032, 10116, 10220, 10303, 10420, 10523, 10601, 10628, 10673, 10774, 10803, 10955, 10967, 11110, 11132, 11159, 11166, 11429, 11657, 11676, 11753, 11889, 11919, 11930, 12112, 12232, 12387, 12566, 12583, 12949, 13020, 13074, 13081, 13256, 13481, 13491, 13533, 13628, 13688, 13693, 13722, 13723, 13775, 13781, 13824, 13920, 13981, 14231, 14296, 14339, 14385, 14466, 14540, 14770, 14892, 14921]


In [ ]:
mc_df.drop(mc_df.index[index_drop], inplace = True)

In [ ]:
print(mc_df)

             id  asthma  heart_failure  heart_disease  angina  heart_attack  \
0      109263.0     2.0            0.0            0.0     0.0           0.0   
1      109264.0     2.0            0.0            0.0     0.0           0.0   
2      109265.0     2.0            0.0            0.0     0.0           0.0   
3      109266.0     2.0            2.0            2.0     2.0           2.0   
4      109267.0     2.0            2.0            2.0     2.0           2.0   
...         ...     ...            ...            ...     ...           ...   
14981  124818.0     2.0            2.0            2.0     2.0           2.0   
14982  124819.0     2.0            0.0            0.0     0.0           0.0   
14983  124820.0     2.0            0.0            0.0     0.0           0.0   
14984  124821.0     1.0            2.0            2.0     2.0           2.0   
14985  124822.0     2.0            2.0            2.0     2.0           1.0   

       stroke  thyroid  liver  gallbladder  
0     

In [ ]:
mc_df['asthma'] = mc_df['asthma'].replace({2: 0})
mc_df['heart_failure'] = mc_df['heart_failure'].replace({2: 0})
mc_df['heart_disease'] = mc_df['heart_disease'].replace({2: 0})
mc_df['angina'] = mc_df['angina'].replace({2: 0})
mc_df['heart_attack'] = mc_df['heart_attack'].replace({2: 0})
mc_df['stroke'] = mc_df['stroke'].replace({2: 0})
mc_df['thyroid'] = mc_df['thyroid'].replace({2: 0})
mc_df['liver'] = mc_df['liver'].replace({2: 0})
mc_df['gallbladder'] = mc_df['gallbladder'].replace({2: 0})
mc_df['heart_attack'] = mc_df['heart_attack'].replace({2: 0})

mc_df.head()

,id,asthma,heart_failure,heart_disease,angina,heart_attack,stroke,thyroid,liver,gallbladder
0,109263.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,109264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,109265.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,109266.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,109267.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mc_df.shape

(14848, 10)

In [ ]:
X = mc_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina,stroke,thyroid,liver,gallbladder
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y = mc_df['heart_attack'].values.reshape(-1,1)
y[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=444, stratify = y)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# creating random grid
n_estimators = [int(x) for x in np.linspace(200, 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid,
                               n_iter= 100, cv = 5, verbose = 2, random_state = 444, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=444, verbose=2)

In [ ]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

In [ ]:
best_random = rf_random.best_estimator_
predictions = best_random.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, predictions)

In [ ]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,3594,15
Actual 1,76,27


0.9754849137931034
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3609
         1.0       0.64      0.26      0.37       103

    accuracy                           0.98      3712
   macro avg       0.81      0.63      0.68      3712
weighted avg       0.97      0.98      0.97      3712



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [90, 100, 110, 120],
    'max_features': ['auto'],
    'min_samples_leaf': [4, 5, 6],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [200, 300, 400, 1000]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                           cv = 5, n_jobs=-1, verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

grid_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ut

{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [ ]:
best_grid = grid_search.best_estimator_

grid_predictions = best_grid.predict(X_test)

cm = confusion_matrix(y_test, grid_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, grid_predictions)

In [ ]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,3598,11
Actual 1,83,20


0.974676724137931
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      3609
         1.0       0.64      0.26      0.37       103

    accuracy                           0.98      3712
   macro avg       0.81      0.63      0.68      3712
weighted avg       0.97      0.98      0.97      3712



In [ ]:
importances = best_grid.feature_importances_

importances_sorted = sorted(zip(best_grid.feature_importances_, X.columns), reverse = True)
importances_df = pd.DataFrame(importances_sorted)
importances_df

,0,1
0,0.506627,heart_disease
1,0.245863,heart_failure
2,0.131601,angina
3,0.058889,stroke
4,0.023570,gallbladder
5,0.017192,thyroid
6,0.016255,asthma
7,0.000003,liver


baseline reandom forest


In [3]:
med_df = pd.read_csv('/content/med_conditions.csv')

In [5]:
combined_df = med_df[['id', 'asthma', 'heart_failure', 'heart_disease', 'angina_pectoris', 'stroke', 'thyroid', 'liver_condition', 'heart_attack']]
combined_df.head()

,id,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition,heart_attack
0,109266.0,0,0,0,0,0,0,0,0
1,109267.0,0,0,0,0,0,0,0,0
2,109271.0,1,0,0,0,0,0,0,0
3,109273.0,0,0,0,0,0,0,0,0
4,109274.0,0,0,0,0,0,0,0,0


In [6]:
combined_df.columns

Index(['id', 'asthma', 'heart_failure', 'heart_disease', 'angina_pectoris',
       'stroke', 'thyroid', 'liver_condition', 'heart_attack'],
      dtype='object')

In [7]:
combined_df.shape

(9088, 9)

In [8]:
X = combined_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [9]:
y = combined_df['heart_attack'].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=444, stratify = y)

In [11]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


<ipython-input-11-efede872133b>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestClassifier()

In [12]:
predictions = rf.predict(X_test)

In [13]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, predictions)

In [14]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,2149,20
Actual 1,70,33


0.960387323943662
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2169
           1       0.62      0.32      0.42       103

    accuracy                           0.96      2272
   macro avg       0.80      0.66      0.70      2272
weighted avg       0.95      0.96      0.95      2272



down sampled data set

In [58]:
heart_attack = combined_df[combined_df['heart_attack'] == 1]
heart_attack.shape

(412, 9)

In [59]:
non_heart_attack = combined_df[combined_df['heart_attack'] == 0]
non_heart_attack.head()

,id,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition,heart_attack
0,109266.0,0,0,0,0,0,0,0,0
1,109267.0,0,0,0,0,0,0,0,0
2,109271.0,1,0,0,0,0,0,0,0
3,109273.0,0,0,0,0,0,0,0,0
4,109274.0,0,0,0,0,0,0,0,0


In [60]:
non_heart_attack = non_heart_attack.sample(n=824,random_state = 444)
non_heart_attack.head()
non_heart_attack.shape

(824, 9)

In [61]:
down_sampled_df = pd.concat([non_heart_attack, heart_attack])
down_sampled_df.head()
down_sampled_df.shape

(1236, 9)

In [62]:
X = down_sampled_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition
1092,1,0,0,1,0,1,0
5795,0,0,0,0,0,0,0
1195,0,0,0,0,0,0,1
3793,0,0,0,0,0,0,0
2507,0,0,0,0,0,0,0


In [63]:
y = down_sampled_df['heart_attack'].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=444, stratify = y)

In [65]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# creating random grid
n_estimators = [int(x) for x in np.linspace(200, 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

In [66]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid,
                               n_iter= 100, cv = 5, verbose = 2, random_state = 444, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=444, verbose=2)

In [67]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': True}

In [69]:
small_best_random = rf_random.best_estimator_
small_predictions = best_random.predict(X_test)

In [70]:
cm = confusion_matrix(y_test, small_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, small_predictions)

In [71]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, small_predictions))

,Predicted 0,Predicted 1
Actual 0,199,7
Actual 1,30,73


0.8802588996763754
              precision    recall  f1-score   support

           0       0.87      0.97      0.91       206
           1       0.91      0.71      0.80       103

    accuracy                           0.88       309
   macro avg       0.89      0.84      0.86       309
weighted avg       0.88      0.88      0.88       309



In [72]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80 , 90],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [9, 10, 11],
    'n_estimators': [700, 800, 900, 1000]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                           cv = 5, n_jobs=-1, verbose=2)

In [73]:
grid_search.fit(X_train, y_train)

grid_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 9,
 'n_estimators': 900}

In [74]:
small_best_grid = grid_search.best_estimator_

small_grid_predictions = small_best_grid.predict(X_test)

cm = confusion_matrix(y_test, small_grid_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, small_grid_predictions)

In [75]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, small_grid_predictions))

,Predicted 0,Predicted 1
Actual 0,195,11
Actual 1,18,85


0.9061488673139159
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       206
           1       0.89      0.83      0.85       103

    accuracy                           0.91       309
   macro avg       0.90      0.89      0.89       309
weighted avg       0.91      0.91      0.91       309



In [76]:
importances = small_best_grid.feature_importances_

importances_sorted = sorted(zip(small_best_grid.feature_importances_, X.columns), reverse = True)
importances_df = pd.DataFrame(importances_sorted)
importances_df

,0,1
0,0.454073,heart_disease
1,0.202223,heart_failure
2,0.168714,stroke
3,0.128827,angina_pectoris
4,0.022186,thyroid
5,0.014310,asthma
6,0.009667,liver_condition


Down sample - bigger size

In [77]:
heart_attack = combined_df[combined_df['heart_attack'] == 1]
non_heart_attack  = combined_df[combined_df['heart_attack'] == 0]

In [78]:
non_heart_attack = non_heart_attack.sample(n=1236,random_state = 444)
non_heart_attack.shape

(1236, 9)

In [79]:
down_sampled_df = pd.concat([non_heart_attack, heart_attack])
down_sampled_df.head()
down_sampled_df.shape

(1648, 9)

In [80]:
X = down_sampled_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition
1092,1,0,0,1,0,1,0
5795,0,0,0,0,0,0,0
1195,0,0,0,0,0,0,1
3793,0,0,0,0,0,0,0
2507,0,0,0,0,0,0,0


In [81]:
y = down_sampled_df['heart_attack'].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=444, stratify = y)

In [83]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# creating random grid
n_estimators = [int(x) for x in np.linspace(200, 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

In [84]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid,
                               n_iter= 100, cv = 5, verbose = 2, random_state = 444, n_jobs = -1)

rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=444, verbose=2)

In [85]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 30,
 'bootstrap': False}

In [86]:
big_best_random = rf_random.best_estimator_
big_predictions = big_best_random.predict(X_test)

In [87]:
cm = confusion_matrix(y_test, big_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, big_predictions)

In [88]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, big_predictions))

,Predicted 0,Predicted 1
Actual 0,296,13
Actual 1,36,67


0.8810679611650486
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       309
           1       0.84      0.65      0.73       103

    accuracy                           0.88       412
   macro avg       0.86      0.80      0.83       412
weighted avg       0.88      0.88      0.88       412



In [89]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [False],
    'max_depth': [90, 100, 110, 120],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [1, 2, 3],
    'n_estimators': [100, 200, 300, 1000]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                           cv = 5, n_jobs=-1, verbose=2)

In [90]:
grid_search.fit(X_train, y_train)

grid_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
240 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ut

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 300}

In [91]:
big_best_grid = grid_search.best_estimator_

big_grid_predictions = big_best_grid.predict(X_test)

cm = confusion_matrix(y_test, big_grid_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y_test, big_grid_predictions)

In [92]:
display(cm_df)
print(acc_score)
print(classification_report(y_test, big_grid_predictions))

,Predicted 0,Predicted 1
Actual 0,296,13
Actual 1,36,67


0.8810679611650486
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       309
           1       0.84      0.65      0.73       103

    accuracy                           0.88       412
   macro avg       0.86      0.80      0.83       412
weighted avg       0.88      0.88      0.88       412



In [93]:
importances = big_best_grid.feature_importances_

importances_sorted = sorted(zip(big_best_grid.feature_importances_, X.columns), reverse = True)
importances_df = pd.DataFrame(importances_sorted)
importances_df

,0,1
0,0.452400,heart_disease
1,0.251602,heart_failure
2,0.135226,angina_pectoris
3,0.114821,stroke
4,0.017339,thyroid
5,0.015058,asthma
6,0.013553,liver_condition


Applying Model to Whole Data to check for biases

In [94]:
X = combined_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [95]:
y = combined_df['heart_attack'].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [102]:
small_total_predictions = small_best_grid.predict(X)
big_total_predictions = big_best_grid.predict(X)

In [97]:
cm = confusion_matrix(y, small_total_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y, small_total_predictions)

In [98]:
display(cm_df)
print(acc_score)
print(classification_report(y, small_total_predictions))

,Predicted 0,Predicted 1
Actual 0,7983,693
Actual 1,85,327


0.9143926056338029
              precision    recall  f1-score   support

           0       0.99      0.92      0.95      8676
           1       0.32      0.79      0.46       412

    accuracy                           0.91      9088
   macro avg       0.66      0.86      0.71      9088
weighted avg       0.96      0.91      0.93      9088



In [103]:
cm = confusion_matrix(y, big_total_predictions)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y, big_total_predictions)

In [104]:
display(cm_df)
print(acc_score)
print(classification_report(y, big_total_predictions))

,Predicted 0,Predicted 1
Actual 0,8364,312
Actual 1,137,275


0.9505941901408451
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8676
           1       0.47      0.67      0.55       412

    accuracy                           0.95      9088
   macro avg       0.73      0.82      0.76      9088
weighted avg       0.96      0.95      0.95      9088



Testing on 2015-2016 Data 

In [105]:
med_conditions_2015_2016 = pd.read_csv('/content/med_conditions_15_16.csv')
med_conditions_2015_2016.head()

,Unnamed: 0,id,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition,heart_attack,asthma_age,heart_failure_age,heart_disease_age,angina_pectoris_age,heart_attack_age,stroke_age,thyroid_age,liver_age,liver
0,0,83732.0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,83733.0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,83734.0,1,0,0,0,0,1,1,1,60.0,NaN,NaN,NaN,58.0,NaN,39.0,11.0,1
3,3,83735.0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,83736.0,1,0,0,0,0,0,0,0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [106]:
mc_df = med_conditions_2015_2016[['id', 'asthma', 'heart_failure', 'heart_disease', 'angina_pectoris', 'stroke', 'thyroid', 'liver_condition', 'heart_attack']]
mc_df.head()

,id,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition,heart_attack
0,83732.0,0,0,0,0,0,0,0,0
1,83733.0,0,0,0,0,0,0,0,0
2,83734.0,1,0,0,0,0,1,1,1
3,83735.0,0,0,0,0,0,0,0,0
4,83736.0,1,0,0,0,0,0,0,0


In [107]:
X = mc_df.copy()
X.drop(['heart_attack', 'id'], axis = 1, inplace = True)
X.head()

,asthma,heart_failure,heart_disease,angina_pectoris,stroke,thyroid,liver_condition
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,1,0,0,0,0,1,1
3,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [108]:
y = mc_df['heart_attack'].values.reshape(-1,1)

In [109]:
small_predictions_15_16 = small_best_grid.predict(X)

In [110]:
cm = confusion_matrix(y, small_predictions_15_16)
cm_df = pd.DataFrame(cm, 
                     index = ["Actual 0", "Actual 1"], columns = ["Predicted 0", "Predicted 1"])

acc_score = accuracy_score(y, small_predictions_15_16)

In [111]:
display(cm_df)
print(acc_score)
print(classification_report(y, small_predictions_15_16))

,Predicted 0,Predicted 1
Actual 0,5043,366
Actual 1,70,162


0.92270873958518
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      5409
           1       0.31      0.70      0.43       232

    accuracy                           0.92      5641
   macro avg       0.65      0.82      0.69      5641
weighted avg       0.96      0.92      0.94      5641

